In [1]:
# libraries to import
import numpy as np
import matplotlib.pyplot as plt

# Stochastic Linear Bandits

# Strategy 1: UCB Algorithm

The difference here is that instead of having confidence intervals, we have confidence ellipsoids.

The goal of the algorithm is to optimize the selection of actions (arms) in order to maximize the cumulative reward received over a series of rounds.

The key idea behind the algorithm is to model the relationship between the observed rewards and the features of each arm using linear regression, and then use the estimated regression coefficients to make informed decisions about which arm to select in each round.

### Steps of the algorithm



In [20]:
# --------- Start of the algorithm LinUCB -----------
def linUCB_algorithm(n_arms, n_features, item_features, true_theta, n_rounds, noise, lambda_param, beta_param, delta_param):
    
    # Initialize feature matrix.
    # V_t[i, j] represents the sum of the product of the ith and jth feature components across all observed data points.
    # V_t[i, i] represents the sum of the squared ith feature components.
    # Initializing V_0 to lambda * Identity matrix makes it invertible.
    V_t = lambda_param * np.eye(n_features)

    # Initialize label vector.
    # Represents the sum of the product of the observed rewards with each feature component across all observed data points.
    sum_A_s_X_s = np.zeros(n_features)
    
    # Initialize theta_hat matrix, where each column represents the estimate of the true_theta vector at each round.
    theta_hat = np.zeros((n_features, n_rounds + 1))
    theta_hat[:, 0] = np.random.uniform(low=0, high=1, size=(n_features, 1)).reshape((n_features,))


    # Initialize arrays to store actions, rewards and regrets.
    actions = np.zeros(n_rounds + 1, dtype=int)
    rewards = np.zeros(n_rounds + 1)
    regrets = np.zeros(n_rounds + 1)
    
    print("theta at time step 0 : ")
    print(theta_hat[:, 0])

    for t in range(1, n_rounds + 1):
        
        # Compute beta parameter for this round using the given formula.
        beta_param_t = 2 * np.log(t * 2) # 2 * np.log(t * n_arms * lambda_param / delta_param)
        print("beta param : " + str(beta_param_t))
        
        # Choose the best action based on the last theta_hat.
        max_value = -np.Inf
        max_index = -1
        for i in range(n_arms):
            # Compute the UCB for each arm using the given formula.
            value = theta_hat[:, t-1].T @ item_features[:, i] + np.sqrt(beta_param_t) * np.sqrt(item_features[:, i].T @ np.linalg.inv(V_t) @ item_features[:, i])
            print("Value for arm " + str(i) + " : " + str(value))
            if value >= max_value:
                max_value = value
                max_index = i
        # Update the best action to take for this round.
        actions[t] = max_index
        
        print("best action to take : " + str(actions[t]))
        
        # Observe the reward of the chosen action and add noise.
        rewards[t] = true_theta.T @ item_features[:, actions[t]] + np.random.normal(scale=noise) # this might be the reason why we get negative regret
        
        print("current reward of chosen action : " + str(rewards[t]))
        
        # Compute the regret for this round.
        optimal_reward = np.max(item_features.T @ true_theta) # todo put this outside the for loop
        print("optimal reward possible : " + str(optimal_reward) + " with arm " + str(np.argmax(item_features.T @ true_theta)))
        regret = optimal_reward - rewards[t]
        print("regret : " + str(regret))
        regrets[t] = regrets[t - 1] + regret
        
        # Update the feature matrix V_t by adding the outer product of the chosen action's feature vector with itself.
        V_t += item_features[:, actions[t]] @ item_features[:, actions[t]].T 
        
        # Compute the inverse of the updated feature matrix V_t.
        V_t_inv = np.linalg.inv(V_t)

        # Update the label vector sum_A_s_X_s by adding the outer product of the chosen action's feature vector with the observed reward.
        sum_A_s_X_s += item_features[:, actions[t]] * rewards[t]
        
        # Compute the new estimate of the true_theta vector using the updated feature matrix and label vector.
        # This estimate represents the center of the ellipsoid in the feature space.
        # theta_hat_not_normalized = V_t_inv @ sum_A_s_X_s
        # theta_hat[:, t] = V_t_inv @ sum_A_s_X_s # theta_hat_not_normalized / np.linalg.norm(theta_hat_not_normalized) # todo idk if we should normalize this np.linalg.norm()

        theta_hat_not_normalized = V_t_inv @ sum_A_s_X_s
        theta_hat[:, t] = theta_hat_not_normalized / np.linalg.norm(theta_hat_not_normalized)

        
        print("theta at time step " + str(t) + " : ")
        print(theta_hat[:, t])
        
        print(" -------------\n ")
        
    return actions, rewards, regrets, theta_hat[:, n_rounds]


In [ ]:
# Setting up parameters for the runs
n_arms = 10
n_features = 5
item_features = np.random.uniform(low=0, high=1, size=(n_features, n_arms))
true_theta = np.random.uniform(low=0, high=1, size=(n_features, 1))
n_rounds = 1000
noise = 0.0 # 0.1 -> can lead to negative regret since the reward could be the optimal + some noise
beta_param = 6
lambda_param = 0.5
delta_param = 0.1

# Running the algorithm
actions, rewards, regrets, theta_hat = linUCB_algorithm(n_arms, n_features, item_features, true_theta, n_rounds, noise, lambda_param, beta_param, delta_param)

# Plotting the regret
plt.plot(regrets)
plt.xlabel('Round')
plt.ylabel('Cumulative Regret')
plt.show()

# Printing the true theta a
print("The true theta is equal to : " + str(true_theta))
print("The theta hat we find at the end is equal to : " + str(theta_hat / np.linalg.norm(theta_hat)))

theta at time step 0 : 
[0.53501947 0.49396339 0.76195145 0.77022141 0.09948563]
beta param : 1.3862943611198906
Value for arm 0 : 2.7198043770377005
Value for arm 1 : 3.3137928418556175
Value for arm 2 : 4.57664534871338
Value for arm 3 : 2.4263033499617537
Value for arm 4 : 3.655264906911391
Value for arm 5 : 3.609690250072143
Value for arm 6 : 4.466276460495173
Value for arm 7 : 4.482106746384206
Value for arm 8 : 3.9074650210952706
Value for arm 9 : 3.987942672027069
best action to take : 2
current reward of chosen action : 1.619291135301408
optimal reward possible : 1.619291135301408 with arm 2
regret : 0.0
theta at time step 1 : 
[ 0.10488211  0.6896571   0.17615202 -0.69429701  0.01717411]
 -------------
 
beta param : 2.772588722239781
Value for arm 0 : 0.7093937273172896
Value for arm 1 : 1.8844632354161386
Value for arm 2 : 1.8596859127020262
Value for arm 3 : 1.0102304256894301
Value for arm 4 : 1.1375825331128977
Value for arm 5 : 1.688568541912469
Value for arm 6 : 1.35805

In [22]:
print(true_theta)
print(item_features)

[[0.86934449]
 [0.53217908]
 [0.24506324]
 [0.28796185]
 [0.22912931]]
[[0.38934231 0.50495469 0.73280492 0.04126355 0.38782427 0.11250376
  0.83169142 0.64843397 0.92521671 0.94796026]
 [0.35949885 0.29400045 0.98431486 0.3143066  0.6691012  0.78670637
  0.43038155 0.87864098 0.00886487 0.12042883]
 [0.38793051 0.93404792 0.76345789 0.19262319 0.67323461 0.22935695
  0.76788901 0.77397894 0.82998078 0.7479978 ]
 [0.42573917 0.10887464 0.38908043 0.72984894 0.6363685  0.84175676
  0.87648089 0.51889743 0.30292794 0.47356693]
 [0.60771634 0.53765437 0.69508198 0.3791686  0.32649528 0.68238134
  0.24966859 0.67897967 0.74830792 0.64742389]]


In [ ]:
# I will wrap everything in a class when I get a working algorithm

# class UCB_linear_bandit:
#     def __init__(self, true_theta, item_features, num_rounds, noise):
#         self.true_theta = true_theta
#         self.item_features = item_features
#         self.num_rounds = num_rounds
#         self.noise = noise
        
#         self.actions = np.zeros(num_rounds)
#         self.regrets = np.zeros(num_rounds)